# 네이버 스마트스토어 리뷰 크롤러

작성일: 2019-07-04  
작성자: 윤현영  

네이버 스마트스토어에 달린 개별 리뷰 정보를 수집하는 크롤러  
- 작성자
- 작성일
- 별점
- 본문텍스트
- 이미지 유무

In [92]:
from selenium import webdriver 
from bs4 import BeautifulSoup as bs
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

path = '../../../chromedriver'
driver = webdriver.Chrome(path)

## 동적 페이지네이션
리뷰 섹션에서 페이지를 이동해도 url이 바뀌지 않는 것으로 보아 javascript로 리뷰의 각 페이지를 로드하는 것으로 보인다. selenium을 활용해서 페이지를 이동할 필요가 있다.

### Stale Element Reference
참고: https://developer.mozilla.org/en-US/docs/Web/WebDriver/Errors/StaleElementReference  
> When an element is no longer attached to the DOM, i.e. it has been removed from the document or the document has changed, it is said to be stale. The stale element reference error is a WebDriver error that occurs because the referenced web element is no longer attached to the DOM.  

a tag를 `nav = driver.find_elements_by_xpath("//nav[@class='module_pagination _review_list_page']//a")`를 통해서 모두 찾은 뒤 for문을 돌며 `.click()`을 하면 stale element reference error가 발생. 아마 다른 페이지를 클릭하면서 WebElement가 조금씩 바뀌는 듯 하다.(html 코드는 바뀌지 않는데??? 모르겠) 여하튼 페이지 개수를 확인한 후 다시 해당하는 a link를 찾아 클릭하는 코드로 변경.


### 구조가 상이한 smartstore들
페이지네이션 부분 html이 다르다  
https://smartstore.naver.com/mdoc/products/267827669#revw  
https://smartstore.naver.com/scinic/products/393299607  

In [69]:
def get_pages(url):
    '''
    input: 스마트 스토어의 상품 url
    output: 각 리뷰 페이지 soup들의 리스트
    '''
    # url = "https://smartstore.naver.com/cwcorp/products/4546888613?NaPm=ct%3Djxnjuulk%7Cci%3D6329f92d6776aed57df797933f968814ebcab8d3%7Ctr%3Dslsl%7Csn%3D646624%7Cic%3D%7Chk%3Dc0a6e36ec9d8fef3fd7a97f9d0bc53d83dc7239a#revw"

    # open the driver
    driver.get(url)
    driver.implicitly_wait(3)

    # find out the total number of pages
    area = driver.find_elements_by_xpath("//div[@id='area_review_list']")

    # list of html-parsed objects for each page
    pages = []

    for i in range(1, cnt+1): # for all the pages from 1 to the end
        try:
            # find <a> tag for page i
            key = 'goPage({})'.format(i)
            a = driver.find_element_by_xpath("//nav[@class='module_pagination _review_list_page']//a[contains(@class, key)]")

            # move to a certain page. first <a> tag will also be clicked
            a.click() 
            driver.implicitly_wait(3)

            # div wrapping a specific page of reviews
            content = driver.find_element_by_xpath("//div[@class='detail_list_review _review_list']").get_attribute('innerHTML') # 어차피 한 개 밖에 없다
            content = bs(content, 'html.parser')
            pages.append(content)

        except:
            print("FAILED: page {} of url {}".format(a.text, url))

#     driver.quit()
    return pages

## 리뷰 섹션의 상세 정보
`get_pages`로 받아온 `div.detail_list_revie _review_list` 안에 ul 안에 개별 li가 하나의 리뷰에 해당

li.item_review _review_list_item_wrap > div.area_user_review
- string 별점: div.area_star_small > span.number_grade 의 text
- string 작성자: div.area_status_user > 첫번째 span.text_info
- string YYMMDD 작성일: div.area_status_user > 두번째 span.text_info
- string 본문 텍스트: `p.review_text _review_text`
- boolean 이미지 유무: `div.cell_thumbnail _has_thumbnail` 의 유무

In [70]:
def get_reviews_of(page):
    return page.find_all('li', {'class': 'item_review _review_list_item_wrap'})

In [71]:
def get_detail(review):
    star = review.find('span', {'class':'number_grade'}).text
    status = review.find('div', {'class': 'area_status_user'}).find_all('span', {'class':'text_info'})
    user = status[0].text
    date = status[1].text.replace('.', '')
    content = review.find('p', {'class': 'review_text _review_text'}).text
    content = strip(content)
    image = (True if review.find('div', {'class': 'cell_thumbnail _has_thumbnail'}) else False)
    
    try:
        title = content[:100]
    except IndexError:
        title = content
    
    return [star, user, date, title, content, image]

In [72]:
import re

def strip(text):
    '''
    '\n', '\t', '\r', ','를 공백으로 대체
    '''
    return re.sub(r'[\n\r\t,]', ' ', text.strip())

In [86]:
result = [get_detail(review) for page in pages for review in get_reviews_of(page)]

df = pd.DataFrame(result, columns = ['star', 'user', 'date', 'content', 'image'])
# df.to_csv('sample.csv', index = False)
df.head()

,grade,user,date,text,image
0,4,c_vi****,190619,민감피부용으로 구매했어요 전에 쓰던 제형보다 묽고(손등에서 흐를 정도는 아닙니다)...,True
1,5,qjaw****,190625,저희 아버지 생신겸 해서 저도 구매했는데요 저는 민감성 피부라서 폼클린징도 소금기있...,False
2,5,sang****,190702,적당하고 무난해서 좋습니다. 너무 묽지도 않고 향도 자극적이지 않네요~,True
3,5,obse****,190703,아빠 사드렸는데 잘 모르겠다고 하시는데. 나쁘지 않다는 이야기일거에요~ ㅋㅋ많이 파세요~,True
4,4,dl25****,190703,배송이 빨라서 좋네요,True


In [73]:
def review_crawler(url):
    '''
    input: 네이버 스마트스토어 url
    output: 해당 상품의 모든 리뷰를 담은 dataframe. 같은 경로에 csv 파일 저장
    '''
    pages = get_pages(url)
    result = [get_detail(review) for page in pages for review in get_reviews_of(page)]
 
    return result

## 제조사/브랜드 추가하기

In [74]:
import requests 

def item_crawler(url):
#     url = "https://smartstore.naver.com/cwcorp/products/4546888613?NaPm=ct%3Djxnjuulk%7Cci%3D6329f92d6776aed57df797933f968814ebcab8d3%7Ctr%3Dslsl%7Csn%3D646624%7Cic%3D%7Chk%3Dc0a6e36ec9d8fef3fd7a97f9d0bc53d83dc7239a#revw"
    
    driver.get(url)
    component = driver.find_element_by_xpath("//div[@class='goods_component3 _product_basic']").get_attribute('innerHTML')
#     driver.quit()
    
    soup = bs(component, 'html.parser')

    try:
        brand = soup.find("th", string="브랜드").find_next('td').string
    except:
        brand = ""

    try:
        manufacturer = soup.find("th", string="제조사").find_next('td').string
    except:
        manufacturer = ""
        
    return brand, manufacturer

## MAIN
https://medium.com/@rtjeannier/pandas-101-cont-9d061cb73bfc: iterrows 쓰지 말기!!  


In [75]:
urlList = pd.read_csv("UrlList.csv", index_col=0)
urlList.head()

,url
0,http://cr2.shopping.naver.com/adcrNoti.nhn?x=f...
1,http://cr2.shopping.naver.com/adcrNoti.nhn?x=g...
2,http://cr2.shopping.naver.com/adcrNoti.nhn?x=w...
3,http://cr2.shopping.naver.com/adcrNoti.nhn?x=b...
4,http://cr2.shopping.naver.com/adcrNoti.nhn?x=F...


In [22]:
smartstore_items = []
smartstore_reviews = []

for i in urlList.index:
    url = urlList.loc[i, 'url']
    
    # ignore if it's not naver smartstore
    if not url.startswith('https://smartstore.naver.com'):
        continue
    
    ##### item info #####
    brand, manufacturer = item_crawler(url)
    smartstore_items.append([i, brand, manufacturer])
    
    ##### reviews #####
    smartstore_reviews.extend(review_crawer(i, url))
    

https://smartstore.naver.com/mdoc/products/267827669
https://smartstore.naver.com/pgshop/products/100604057
https://smartstore.naver.com/scinic/products/393299607
https://smartstore.naver.com/itcoshop2/products/114499693
https://smartstore.naver.com/makeupi/products/140460458
https://smartstore.naver.com/hicos2010/products/468201809
https://smartstore.naver.com/kocoslab2016/products/498581980
https://smartstore.naver.com/scinic/products/212651244
https://smartstore.naver.com/heisclean/products/368716748
https://smartstore.naver.com/lgcarecm/products/522404922
https://smartstore.naver.com/scinic/products/418657154
https://smartstore.naver.com/minamproject/products/481244648
https://smartstore.naver.com/koreanet/products/292760478
https://smartstore.naver.com/minamproject/products/481245297
https://smartstore.naver.com/soonsushop/products/2023131178
https://smartstore.naver.com/phyto/products/389854589
https://smartstore.naver.com/tosowoong/products/100123032
https://smartstore.naver.com

In [77]:
url = "https://smartstore.naver.com/scinic/products/393299607"
%timeit reviews = review_crawler(url)

4.41 s ± 503 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
reviews = review_crawler(url)

In [81]:
pages = get_pages(url)
pages

[]

In [93]:
# open the driver
driver.get(url)
driver.implicitly_wait(3)

In [94]:
# limit area
area = driver.find_element_by_xpath("//div[@id='area_review_list']")

# list of html-parsed objects for each page
pages = []

In [95]:
# list of html-parsed objects for each page
pages = []
# initialize page counter
page = 1

while True:
    # 현재 페이지 수집
    content = area.find_element_by_xpath("//div[@class='detail_list_review _review_list']").get_attribute('innerHTML') # 어차피 한 개 밖에 없다
    content = bs(content, 'html.parser')
    pages.append(content)
    
    # page +1
    page += 1
    key = 'goPage({})'.format(page)
    
    # 다음 페이지가 존재하는 경우
    if area.find_element_by_xpath("//a[contains(@class, key)]"):
        a = area.find_element_by_xpath("//a[contains(@class, key)]")
        a.click()
        driver.implicitly_wait(3)
        
    # 다음 페이지 세트가 존재하는 경우 (e.g. 방금 10 페이지를 긁었고 이제 11 페이지를 긁어야 할 경우)
    elif area.find_element_by_xpath("//a[contains(@class, 'goNextPageSet()')]"):
        a = area.find_element_by_xpath("//a[contains(@class, 'goNextPageSet()')]")
        a.click()
        driver.implicitly_wait(3)
    
    # 둘 다 아닐 경우
    else:
        break

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (65, -15)
  (Session info: chrome=75.0.3770.100)
